In [3]:
import numpy as np
import pandas as pd
import glob
import seaborn as sn  # for heatmaps
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import glob

# Pending

In [45]:
import numpy as np
import pandas as pd
import glob

def generate_filepaths(fp):
    return fp+"/input*.csv", fp+"/annotation*.csv"

def list_to_string(ls):
    string = ""
    for i in ls:
        string += i
    return string 

def col_generate(exp_no):
    if exp_no == 'exp_1':
        col = []
        for i in range(1,91):
            col.append(str(f'amp_{i}'))
        for i in range(1,91):
            col.append(str(f'theta_{i}'))
    elif exp_no == 'exp_2':
        col = ['time','unknown_1']
        for i in range(1,91):
            col.append(str(f'amp_{i}'))
    elif exp_no == 'exp_3':
        col = ['time','unknown_1']
        for i in range(1,91):
            col.append(str(f'amp_{i}'))
    else:  raise Exception("Error occured in function: col_generate")
    return col

def import_single_file(filepath_x,filepath_y,sample_id,exp_no):
    df = pd.read_csv(filepath_x,names=col_generate(exp_no),header=0)
    df['user'] = sample_id
    df['label'] = pd.read_csv(filepath_y,names=['label'],header=0)
    return df

def import_experimental_data(exp_no,fp):
    searchpaths_x,searchpaths_y = generate_filepaths(fp)
    dataframes = []
    filepaths_x = sorted(glob.glob(searchpaths_x))
    filepaths_y = sorted(glob.glob(searchpaths_y))
    assert len(filepaths_x) == len(filepaths_y)
    print(f"Found {len(filepaths_x)} files.")
    for filepath_x,filepath_y in zip(filepaths_x,filepaths_y):
        user = filepath_x.split('.')[0].split("_")[7:]
        user = list_to_string(user)
        dataframes.append(import_single_file(filepath_x,filepath_y,user,exp_no))
        print(filepath_x.split('\\')[-1],filepath_y.split('\\')[-1],user)
#         assert False
    df = pd.concat(dataframes,axis=0)
    return df


fp = "E://external_data/Experiment1/csv_files/wifi_csi_anechoic_csv"
df = import_experimental_data('exp_3',fp)

Found 3 files.
input_user_user_1.csv annotation_user_user_1.csv user1
input_user_user_2.csv annotation_user_user_2.csv user2
input_user_user_2_2.csv annotation_user_user_2_2.csv user22


In [1]:
import numpy as np
import pandas as pd
import glob

def generate_filepaths(fp):
    return fp+"/input*.csv", fp+"/annotation*.csv"

def list_to_string(ls):
    string = ""
    for i in ls:
        string += i
    return string 

def col_generate(exp_no):
    if exp_no == 'exp_1':
        col = []
        for i in range(1,91):
            col.append(str(f'amp_{i}'))
        for i in range(1,91):
            col.append(str(f'theta_{i}'))
    elif exp_no == 'exp_2':
        col = ['time','unknown_1']
        for i in range(1,91):
            col.append(str(f'amp_{i}'))
    elif exp_no == 'exp_3':
        col = ['time','unknown_1']
        for i in range(1,91):
            col.append(str(f'amp_{i}'))
    else:  raise Exception("Error occured in function: col_generate")
    return col


def import_single_file(filepath_x,filepath_y,sample_id,exp_no):
    df = pd.read_csv(filepath_x,names=col_generate(exp_no),header=0)
    df['user'] = sample_id
    df['label'] = pd.read_csv(filepath_y,names=['label'],header=0)
    return df

def import_experimental_data(exp_no,fp):
    searchpaths_x,searchpaths_y = generate_filepaths(fp)
    dataframes = []
    filepaths_x = sorted(glob.glob(searchpaths_x))
    filepaths_y = sorted(glob.glob(searchpaths_y))
    assert len(filepaths_x) == len(filepaths_y)
    print(f"Found {len(filepaths_x)} files.")
    for filepath_x,filepath_y in zip(filepaths_x,filepaths_y):
        if exp_no in ['exp_1','exp_2']:
            user = filepath_x.split('\\')[-1].split("_")[1]
        elif exp_no == 'exp_3':
            user = filepath_x.split('.')[0].split("_")[7:]
            user = list_to_string(user)
        else:
            raise Exception('Error with exp_no')
        dataframes.append(import_single_file(filepath_x,filepath_y,user,exp_no))
        print(filepath_x.split('\\')[-1],filepath_y.split('\\')[-1],user)
    df = pd.concat(dataframes,axis=0)
    return df

def clean_data(name,df):
    """
    Clean specific dataset based on the name. Available dataset are {"exp1",
    "exp2"}
    """
    if name == 'exp_1':
        pass
        # df = df.reset_index(drop=True)
        # df.user = df.user.apply(lambda x: x.split("\\")[0])
    elif name == 'exp_2':
        #  df = df.reset_index(drop=True)
        df.user = df.user.apply(lambda x: x.split('.')[0])
        df = df.iloc[:,2:]
    elif name == 'exp_3':
        #  df = df.reset_index(drop=True)
        df = df.iloc[:,2:]
        df['label'] = df['label'].fillna('noactivity')
    return df

def import_clean_data(name,fp):
    """
    import clean dataset based on the name. Available dataset are {"exp1",
    "exp2"}
    """
    if name in ['exp_1','exp_2','exp_3']:
        df = import_experimental_data(name,fp)
        df = clean_data(name,df)
    else:
        return
    return df

fp = "E:/external_data/Experiment3/csv_files/exp_1"
df = import_clean_data('exp_1',fp)

Found 80 files.
input_user10_bendfwd.csv annotation_user10_bendfwd.csv user10
input_user10_kneel.csv annotation_user10_kneel.csv user10
input_user10_lie.csv annotation_user10_lie.csv user10
input_user10_sit.csv annotation_user10_sit.csv user10
input_user10_sitrotate.csv annotation_user10_sitrotate.csv user10
input_user10_stand.csv annotation_user10_stand.csv user10
input_user10_standrotate.csv annotation_user10_standrotate.csv user10
input_user10_walking.csv annotation_user10_walking.csv user10
input_user1_bendfwd.csv annotation_user1_bendfwd.csv user1
input_user1_kneel.csv annotation_user1_kneel.csv user1
input_user1_lie.csv annotation_user1_lie.csv user1
input_user1_sit.csv annotation_user1_sit.csv user1
input_user1_sitrotate.csv annotation_user1_sitrotate.csv user1
input_user1_stand.csv annotation_user1_stand.csv user1
input_user1_standrotate.csv annotation_user1_standrotate.csv user1
input_user1_walking.csv annotation_user1_walking.csv user1
input_user2_bendfwd.csv annotation_user2

In [2]:
df

,amp_1,amp_2,amp_3,amp_4,amp_5,amp_6,amp_7,amp_8,amp_9,amp_10,...,theta_83,theta_84,theta_85,theta_86,theta_87,theta_88,theta_89,theta_90,user,label
0,31.675,43.080,53.718,60.338,63.286,63.721,64.516,65.070,66.817,67.594,...,-2.519100,-2.568700,-2.577200,-2.551200,-2.551900,-2.517600,-2.498000,-2.10120,user10,noactivity
1,31.914,43.621,54.341,60.922,64.017,64.344,65.142,65.819,67.606,68.344,...,1.068600,1.048600,1.057300,1.053000,1.103500,1.075600,1.111100,1.44400,user10,noactivity
2,32.180,44.219,55.030,61.563,64.823,65.029,65.835,66.646,68.476,69.172,...,-3.970200,-4.005300,-4.022800,-3.961900,-3.968900,-3.926500,-3.887700,-3.54740,user10,noactivity
3,32.424,44.768,55.661,62.150,65.561,65.659,66.471,67.404,69.277,69.934,...,0.070351,-0.001486,-0.033409,-0.012021,0.029555,0.061227,0.077287,0.40222,user10,noactivity
4,32.669,45.316,56.291,62.737,66.299,66.290,67.107,68.163,70.079,70.696,...,-0.137010,-0.172940,-0.191950,-0.158630,-0.122420,-0.152340,-0.094355,0.26548,user10,noactivity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6994,53.154,64.723,73.336,77.902,78.252,76.008,69.841,64.506,59.615,59.865,...,3.377100,3.324000,3.266000,3.219000,3.182300,3.177700,3.150100,3.46650,user9,walk
6995,53.512,65.181,73.851,78.438,78.779,76.417,70.220,64.849,59.855,60.027,...,3.453500,3.390500,3.354000,3.299400,3.267400,3.233800,3.230300,3.55240,user9,walk
6996,54.134,65.965,74.763,79.381,79.709,77.317,71.098,65.684,60.560,60.637,...,3.234300,3.199600,3.131600,3.087600,3.051100,3.037700,3.008000,3.32310,user9,walk
6997,54.894,66.922,75.882,80.545,80.854,78.481,72.244,66.789,61.511,61.482,...,3.560300,3.510200,3.449200,3.389700,3.367800,3.344000,3.331100,3.66490,user9,walk


In [31]:
from os import listdir
from os.path import isfile, join
import os
path = "E:/external_data/Experiment2/spectrogram_data_by_activity_csv"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]


In [ ]:
x = [os.path.join(r,file) for r,d,f in os.walk(path) for file in f]
x

In [33]:
[item for item in os.listdir(path)] 

['kneel', 'lie', 'pickup', 'sit', 'stand', 'standff', 'walk']

In [36]:
'input_user1_kneel_2.csv'.split('.')[0].split('_')[1]

'user1'

In [43]:
3 in {1:2,2:3}

False

In [63]:


def import_sepctrograms(path):
    columns = [f"col_{i+1}" for i in range(501)]
    data = dict()
    for label in os.listdir(path):
        data[label] = dict()
        for f in listdir(path+'/'+label):
            user = f.split('.')[0].split('_')[1]
            df = pd.read_csv(path+'/'+label+'/'+f, names=columns , header=0)
            if user not in data[label]:
                data[label][user] = []
            data[label][user].append(df)
        print(label)
    return data

path = "E:/external_data/Experiment2/spectrogram_data_by_activity_csv"
data = import_sepctrograms(path)

kneel
lie
pickup
sit
stand
standff
walk


In [65]:
len(data['kneel']['user1'])

9